In [ ]:
### IMPORTANT: The following path needs to be appended so that the neccessary codes can be read from that directory ###
import sys
sys.path.append('/data1/home/basadieh/behzadcodes/behzadlibrary')
#from Behzadlib import * ### This is loaded from '/data1/home/basadieh/behzadcodes/behzadlibrary'
#from CMIP5lib import *  ### This is loaded from '/data1/home/basadieh/behzadcodes/behzadlibrary'
from Behzadlib import func_MLD, func_time_depth_plot, func_stream, func_ENSO, func_NAO, func_latlon_regrid, func_latlon_regrid_eq, func_oceanlandmask, func_oceanindex, func_regrid, func_plot, func_plot_save, func_plot_bounds_save
from BehzadlibPlot import func_plotmap_contourf, func_plot2Dcolor_contourf, func_plotline_1var, func_plotline_2var, func_plot2Dcontour_1var, func_plot2Dcontour_2var, func_plot_lagcor_sig, func_plot_laggedmaps
from Behzadlib2 import func_barotropicstream_Vx_DyIntegrated, func_barotropicstream_Vy_DxIntegrated, func_MLD_AllYears_annual_ave, func_MLD_AllYears_months_ave, func_MLD_AllYears_1month
from CMIP5lib import netcdf_read, runningMeanFast, plot_PSD_welch, lag_cor, lag_cor_data, calc_Atl_Mask
########################################
import numpy as np
from numpy import zeros, ones, empty, nan, shape
from numpy import isnan, nanmean, nanmax, nanmin
import numpy.ma as ma
from netCDF4 import MFDataset, Dataset, num2date, date2num, date2index
import os
import matplotlib
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm, maskoceans
from scipy.interpolate import griddata
import math
import copy
########################################

dir_pwd = os.getcwd() # Gets the current directory (and in which the code is placed)
dir_figs = (dir_pwd + '/Figures1/') # Directory to save figures
dir_results = (dir_pwd + '/Results/') # Directory to save results

GCM = 'GFDL-ESM2G'
year_start=1
year_end=500

lat_n_regrid, lon_n_regrid = 180, 360 # Number of Lat and Lon elements in the regridded data
lon_min_regrid, lon_max_regrid = 0, 360 # Min and Max value of Lon in the regridded data
lat_min_regrid, lat_max_regrid = -90, 90 # Min and Max value of Lat in the regridded data

Lat_regrid_1D, Lon_regrid_1D, Lat_bound_regrid, Lon_bound_regrid = func_latlon_regrid_eq(lat_n_regrid, lon_n_regrid, lat_min_regrid, lat_max_regrid, lon_min_regrid, lon_max_regrid)
Lon_regrid_2D, Lat_regrid_2D = np.meshgrid(Lon_regrid_1D, Lat_regrid_1D)

Ocean_Land_mask = func_oceanlandmask(Lat_regrid_2D, Lon_regrid_2D) # 1= ocean, 0= land

Ocean_Index = func_oceanindex (Lat_regrid_2D, Lon_regrid_2D) # [0=land] [2=Pacific] [3=Indian Ocean] [6=Atlantic] [10=Arctic] [8=Baffin Bay (west of Greenland)] [9=Norwegian Sea (east of Greenland)] [11=Hudson Bay (Canada)] 

##############################################################################
#################        To restore:        ##################################
import os
import shelve

filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data

my_shelf = shelve.open(filename_out)
for key in my_shelf:
    globals()[key]=my_shelf[key]
my_shelf.close()
dir_figs = (dir_pwd + '/Figures1/') # Directory to save figures

###################################################################
#np.save('WS_index_time_depth_GFDL-ESM2G_500yr.npy',WS_index_time_depth)
##############################################################################
##############################################################################

# MLD_years_monthlyave = np.load('MLD_years_monthlyave_GFDL-ESM2G_500yr.npy')

########                  Sea Ice Concentration                  ###########
t_frequency='OImon'
variable_sic='sic' # Sea Ice Concentration

dir_data_in1 = ('/data5/scratch/CMIP5/CMIP5_models/ocean_physics/') # Directory to raed raw data from
dir_data_in2=(dir_data_in1+ GCM + '/piControl/mo/')

dset_sic = netcdf_read (dir_data_in2+str(variable_sic)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM), variable_sic) # Specifies the netCDF dataset (consisting of multiple files) from which the data will be read
start_date_i,end_date_i = dset_sic.find_time(dset_sic.times, year_start, year_end)

#SIC_Annual_allyears = np.zeros(( int((end_date_i+1-start_date_i)/12), Lat_regrid_2D.shape[0], Lon_regrid_2D.shape[1]))
SIC_Annual_allyears=np.full([int((end_date_i+1-start_date_i)/12),Lat_regrid_2D.shape[0],Lon_regrid_2D.shape[1]], np.nan)
for t in range(int((end_date_i+1-start_date_i)/12)):
    if t%10==0: # Just showing every 10 year
        print('SIC calc - Year: ', year_start+t)
    data_sic_extracted=dset_sic.extract_data(dset_sic.variable, start_date_i+12*t+1-1,start_date_i+12*t+12-1)
    data_sic_extracted=np.nanmean(data_sic_extracted, axis=0)  
    data_sic_extracted=np.squeeze(data_sic_extracted)
    data_i = func_regrid(data_sic_extracted, dset_sic.y, dset_sic.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>1e19]=np.nan
    
    SIC_Annual_allyears[t,:,:]=data_i

#np.save('SIC_Annual_allyears_GFDL-ESM2G_500yr.npy', SIC_Annual_allyears)

fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean(SIC_Annual_allyears,axis=0),30,latlon=True, cmap=plt.cm.jet)
plt.colorbar(im)
plt.title('Annual-average Sea Ice Concentration - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_SIC_mean_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')
###############################################################################

SIC_max_allyears=np.full([int((end_date_i+1-start_date_i)/12),Lat_regrid_2D.shape[0],Lon_regrid_2D.shape[1]], np.nan)
SIC_min_allyears=np.full([int((end_date_i+1-start_date_i)/12),Lat_regrid_2D.shape[0],Lon_regrid_2D.shape[1]], np.nan)
for t in range(int((end_date_i+1-start_date_i)/12)):
    if t%10==0: # Just showing every 10 year
        print('SIC calc - Year: ', year_start+t)
    data_sic_extracted=dset_sic.extract_data(dset_sic.variable, start_date_i+12*t+1-1,start_date_i+12*t+12-1)
    
    data_sic_extracted_1=np.nanmax(data_sic_extracted, axis=0)  
    data_sic_extracted_1=np.squeeze(data_sic_extracted_1)
    data_i = func_regrid(data_sic_extracted_1, dset_sic.y, dset_sic.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>1e19]=np.nan
    SIC_max_allyears[t,:,:]=data_i

    data_sic_extracted_2=np.nanmin(data_sic_extracted, axis=0)  
    data_sic_extracted_2=np.squeeze(data_sic_extracted_2)
    data_i = func_regrid(data_sic_extracted_2, dset_sic.y, dset_sic.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>1e19]=np.nan
    SIC_min_allyears[t,:,:]=data_i


fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean(SIC_max_allyears,axis=0),30,latlon=True, cmap=plt.cm.jet)
plt.colorbar(im)
plt.title('Annual-Maximum Sea Ice Concentration - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
fig.savefig(dir_figs+str(GCM)+'_average_SIC_max_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean(SIC_min_allyears,axis=0),30,latlon=True, cmap=plt.cm.jet)
plt.colorbar(im)
plt.title('Annual-Mainimum Sea Ice Concentration - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
fig.savefig(dir_figs+str(GCM)+'_average_SIC_min_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

###############################################################################
###############################################################################
SST_allyears_ave = np.load('Temp_allyears_GFDL-ESM2G_500yr.npy')
SST_allyears_ave = np.nanmean(np.nanmean(SST_allyears_ave, axis=1), axis=0)
SST_allyears_ave_SO = copy.deepcopy(SST_allyears_ave)
SST_allyears_ave_SO[90:,:] = np.nan


fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,SST_allyears_ave_SO-273.15,np.linspace(-2,24,27),latlon=True, cmap=plt.cm.jet, extend='both')
plt.colorbar(im)
plt.title('Annual-average SST [C] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_SST_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,SST_allyears_ave_SO-273.15,np.linspace(-2,24,27),latlon=True, cmap=plt.cm.jet, extend='both')
plt.colorbar(im)

im2=m.contour(Lon_regrid_2D[0:90,:], Lat_regrid_2D[0:90,:],np.nanmean(SIC_max_allyears,axis=0)[0:90,:], levels = [15], latlon=True, colors='r', linewidth=3.0)
im3=m.contour(Lon_regrid_2D[0:90,:], Lat_regrid_2D[0:90,:],np.nanmean(SIC_min_allyears,axis=0)[0:90,:], levels = [15], latlon=True, colors='r', linewidth=3.0)

plt.title('Annual-average SST [C] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)+'\n Contour = Max & Min Sea Ice extent', fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_SST_SIC_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################

t_frequency='Omon'
variable_thetao='thetao' # Sea Water Temperature
variable_so='so' # Sea Water Salinity

dir_data_in1 = ('/data5/scratch/CMIP5/CMIP5_models/ocean_physics/') # Directory to raed raw data from
dir_data_in2=(dir_data_in1+ GCM + '/piControl/mo/')

dset_thetao = netcdf_read (dir_data_in2+str(variable_thetao)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM), variable_thetao) # Specifies the netCDF dataset (consisting of multiple files) from which the data will be read
dset_so = netcdf_read (dir_data_in2+str(variable_so)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM), variable_so)
Depths=dset_thetao.lvl[:]

month_no=9 # args[2] in main code # month of the year, 9=September, 3=March
MLD_years_september = func_MLD_AllYears_1month(dset_thetao, dset_so, month_no, year_start, year_end, 180, 360)
np.save('MLD_years_september_GFDL-ESM2G_500yr.npy',MLD_years_september)
#MLD_years_september=np.load('MLD_years_september_GFDL-ESM2G_500yr.npy')
dset_thetao.close_ncfile(dset_thetao.fin)
dset_so.close_ncfile(dset_so.fin)




fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean( MLD_years_september, axis=0),np.linspace(0,2400,25),latlon=True, cmap=plt.cm.jet, extend='both')
plt.colorbar(im)
plt.title('September Mixed Layer Depth [m] - Average of all 500 years \n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_MLD_sept_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean( MLD_years_september, axis=0),np.linspace(0,2400,25),latlon=True, cmap=plt.cm.jet, extend='both')
plt.colorbar(im)
m.scatter(lon_mld_WS[WS_indeces_lonlat],lat_mld_WS[WS_indeces_lonlat],2,c='k',latlon=True)
plt.title('September Mixed Layer Depth [m] - Average of all 500 years \n Dots represent the convetion area - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_MLD_sept_SO_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


#Lat_regrid_1D_2, Lon_regrid_1D_2, Lat_bound_regrid_2, Lon_bound_regrid_2 = func_latlon_regrid_eq(90, 180, lat_min_regrid, lat_max_regrid, lon_min_regrid, lon_max_regrid)
#Lon_regrid_2D_2, Lat_regrid_2D_2 = np.meshgrid(Lon_regrid_1D_2, Lat_regrid_1D_2)

fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(Lon_regrid_2D,Lat_regrid_2D,np.nanmean( MLD_years_september, axis=0),np.linspace(0,3200,33),latlon=True, cmap=plt.cm.jet, extend='both')
plt.colorbar(im)
im2=m.contour(lon_mld_WS,lat_mld_WS,MLD_average_WS, levels = [2200], latlon=True, colors='r', linewidth=3.0)

plt.title('September Mixed Layer Depth [m] - Average of all 500 years \n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_MLD_sept_SO.png', format='png', dpi=300, transparent=True, bbox_inches='tight')





fig=plt.figure()
m = Basemap( projection='spstere',lon_0=180,boundinglat=-30)
m.drawcoastlines(linewidth=1.25)
m.fillcontinents(color='0.8')
m.drawparallels(np.arange(-90,90,20), labels=[1,1,0,1])
m.drawmeridians(np.arange(0,360,30), labels=[1,1,0,1])
im=m.contourf(lon_mld_WS,lat_mld_WS,MLD_average_WS,np.linspace(0,4700,48),latlon=True, cmap=plt.cm.jet,  extend='both')
plt.colorbar(im)
m.scatter(lon_mld_WS[WS_indeces_lonlat],lat_mld_WS[WS_indeces_lonlat],2,latlon=True)
plt.title('September Mixed Layer Depth [m] - Convection-Years-Only Average \n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_average_MLD_convection_years.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
import os; import shelve

filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['Depths']=my_shelf['Depths']; my_shelf.close()
#########################################

Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')

Salinity_time_lat_atl_surf=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,0,:,:]
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Salinity_time_lat_atl_surf.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_atl_surf = np.asarray(Salinity_time_lat_atl_surf)

Salinity_time_lat_atl_subsurf=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Salinity_time_lat_atl_subsurf.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_atl_subsurf = np.asarray(Salinity_time_lat_atl_subsurf)

Salinity_time_lat_atl_deep=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Salinity_time_lat_atl_deep.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_atl_deep = np.asarray(Salinity_time_lat_atl_deep)

Salinity_time_lat_atl_deeper=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Salinity_time_lat_atl_deeper.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_atl_deeper = np.asarray(Salinity_time_lat_atl_deeper)

########################################
Salinity_time_lat_pacfc_surf=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,0,:,:]
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Salinity_time_lat_pacfc_surf.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_pacfc_surf = np.asarray(Salinity_time_lat_pacfc_surf)

Salinity_time_lat_pacfc_subsurf=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Salinity_time_lat_pacfc_subsurf.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_pacfc_subsurf = np.asarray(Salinity_time_lat_pacfc_subsurf)

Salinity_time_lat_pacfc_deep=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Salinity_time_lat_pacfc_deep.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_pacfc_deep = np.asarray(Salinity_time_lat_pacfc_deep)

Salinity_time_lat_pacfc_deeper=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Salinity_time_lat_pacfc_deeper.append(np.nanmean(data_i,axis=1))
    
Salinity_time_lat_pacfc_deeper = np.asarray(Salinity_time_lat_pacfc_deeper)

del Salinity_allyears

###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Salinity_time_lat_atl_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Salinity [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_atl_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Salinity [150-600m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_atl_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Salinity [1000-4000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_atl_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Salinity [2000-5000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Salinity_Anomalies_Atlantic.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Salinity_time_lat_pacfc_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Salinity [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_pacfc_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Salinity [150-600m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_pacfc_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Salinity [1000-4000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Salinity_time_lat_pacfc_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Salinity [2000-5000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Salinity_Anomalies_Pacific.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

###############################################################################
###############################################################################
###############################################################################
###############################################################################

Temp_allyears=np.load('Temp_allyears_GFDL-ESM2G_500yr.npy')

Temp_time_lat_atl_surf=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,0,:,:]
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Temp_time_lat_atl_surf.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_atl_surf = np.asarray(Temp_time_lat_atl_surf)

Temp_time_lat_atl_subsurf=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Temp_time_lat_atl_subsurf.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_atl_subsurf = np.asarray(Temp_time_lat_atl_subsurf)

Temp_time_lat_atl_deep=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Temp_time_lat_atl_deep.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_atl_deep = np.asarray(Temp_time_lat_atl_deep)

Temp_time_lat_atl_deeper=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Temp_time_lat_atl_deeper.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_atl_deeper = np.asarray(Temp_time_lat_atl_deeper)

########################################
Temp_time_lat_pacfc_surf=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,0,:,:]
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Temp_time_lat_pacfc_surf.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_pacfc_surf = np.asarray(Temp_time_lat_pacfc_surf)

Temp_time_lat_pacfc_subsurf=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Temp_time_lat_pacfc_subsurf.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_pacfc_subsurf = np.asarray(Temp_time_lat_pacfc_subsurf)

Temp_time_lat_pacfc_deep=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Temp_time_lat_pacfc_deep.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_pacfc_deep = np.asarray(Temp_time_lat_pacfc_deep)

Temp_time_lat_pacfc_deeper=[] 
for t in range(Temp_allyears.shape[0]):
    data_i=Temp_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Temp_time_lat_pacfc_deeper.append(np.nanmean(data_i,axis=1))
    
Temp_time_lat_pacfc_deeper = np.asarray(Temp_time_lat_pacfc_deeper)

del Temp_allyears

###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Temp_time_lat_atl_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Temperature [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_atl_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Temperature [150-600m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_atl_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Temperature [1000-4000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_atl_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Temperature [2000-5000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Temperature_Anomalies_Atlantic.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Temp_time_lat_pacfc_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Temperature [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_pacfc_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Temperature [150-600m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_pacfc_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Temperature [1000-4000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Temp_time_lat_pacfc_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Temperature [2000-5000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Temperature_Anomalies_Pacific.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
###############################################################################
###############################################################################

Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

Density_time_lat_atl_surf=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,0,:,:]
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Density_time_lat_atl_surf.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_atl_surf = np.asarray(Density_time_lat_atl_surf)

Density_time_lat_atl_subsurf=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Density_time_lat_atl_subsurf.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_atl_subsurf = np.asarray(Density_time_lat_atl_subsurf)

Density_time_lat_atl_deep=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Density_time_lat_atl_deep.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_atl_deep = np.asarray(Density_time_lat_atl_deep)

Density_time_lat_atl_deeper=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Density_time_lat_atl_deeper.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_atl_deeper = np.asarray(Density_time_lat_atl_deeper)

########################################
Density_time_lat_pacfc_surf=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,0,:,:]
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Density_time_lat_pacfc_surf.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_pacfc_surf = np.asarray(Density_time_lat_pacfc_surf)

Density_time_lat_pacfc_subsurf=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,14:32,:,:] #150-600m (145-617m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Density_time_lat_pacfc_subsurf.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_pacfc_subsurf = np.asarray(Density_time_lat_pacfc_subsurf)

Density_time_lat_pacfc_deep=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,34:47,:,:] # 1000-4000m (1007-4230m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Density_time_lat_pacfc_deep.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_pacfc_deep = np.asarray(Density_time_lat_pacfc_deep)

Density_time_lat_pacfc_deeper=[] 
for t in range(Density_allyears.shape[0]):
    data_i=Density_allyears[t,34:,:,:] # 2000-5000m (2045-5316m)
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Density_time_lat_pacfc_deeper.append(np.nanmean(data_i,axis=1))
    
Density_time_lat_pacfc_deeper = np.asarray(Density_time_lat_pacfc_deeper)

del Density_allyears

###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Density_time_lat_atl_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Density [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_atl_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Density [150-600m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_atl_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Density [1000-4000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_atl_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Density [2000-5000m] [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Density_Anomalies_Atlantic.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
fig=plt.figure()

Var_plot = Density_time_lat_pacfc_surf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of surface Density [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_pacfc_subsurf
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of subsurface Density [150-600m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_pacfc_deep
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,3)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Density [1000-4000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = Density_time_lat_pacfc_deeper
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(4,1,4)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), 40, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of deep ocean Density [2000-5000m] [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, top=0.9, wspace=0.3, hspace=0.7)
#fig.savefig(dir_figs+str(GCM)+'_Density_Anomalies_Pacific.png', format='png', dpi=300, transparent=True, bbox_inches='tight')





###############################################################################
###############################################################################
###############################################################################
############################################################
########       Sea level pressure composites      ##########
t_frequency='Amon'
variable='psl'

dir_data_in1 = ('/data5/scratch/CMIP5/CMIP5_models/atmosphere_physics/') # Directory to raed raw data from
dir_data_in2=(dir_data_in1+ GCM + '/piControl/mo/')

dset_psl= netcdf_read(dir_data_in2+str(variable)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM),variable)
start_date_i,end_date_i = dset_psl.find_time(dset_psl.times, year_start, year_end)

PSL_final=[]

for i in range(int((end_date_i+1-start_date_i)/12)):
    #print (int((end_date_i+1-start_date_i)/12))
    print('PSL Composte - Year: ', i)
    data_vo_extracted=dset_psl.extract_data(dset_psl.variable,start_date_i+12*i,start_date_i+12*i+11)
    data=np.squeeze(data_vo_extracted)
    data=np.mean(data, axis=0)
    #x_i, y_i = np.meshgrid(dset_ts.y,dset_ts.x)
    #lon,lat,data_i=interpolate_2_reg_grid(dset_ts.x,dset_ts.y,data)
    data_i = func_regrid(data, dset_psl.y, dset_psl.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>10000000]=np.nan
    PSL_final.append(data_i)

PSL_final=np.asarray(PSL_final)
dset_psl.close_ncfile(dset_psl.fin)

PSL_40S=(np.nanmean(PSL_final, axis=2)[:,50] -np.nanmean(np.nanmean(PSL_final, axis=2)[:,50]))/np.std(np.nanmean(PSL_final, axis=2)[:,50])
PSL_65S=(np.nanmean(PSL_final, axis=2)[:,25] -np.nanmean(np.nanmean(PSL_final, axis=2)[:,25]))/np.std(np.nanmean(PSL_final, axis=2)[:,25])
SAM_index = PSL_40S - PSL_65S
SAM_index_rm = runningMeanFast(SAM_index, 10)

#np.save('SAM_index_GFDL-ESM2G_500yr.npy', SAM_index)

#SAM_index = np.nanmean(PSL_final, axis = 2)
#SAM_index = SAM_index[:,50] - SAM_index[:,25] # SAM = zonally averaged PSL at 40S minus 65S
#SAM_index_rm = runningMeanFast(SAM_index, 10)

years=np.linspace(year_start,year_end,year_end-year_start+1)
P_Var_x=years[:-9]
P_Var_y=SAM_index_rm[:-9]
P_xlable='Years'
P_ylable='SAM index'
P_title='SAM index [PSL_40S minus PSL_65S] [Normalized ((value-mean)/std) , smoothed] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)
fig = func_plotline_1var(P_Var_x, P_Var_y, P_xlable, P_ylable, P_title, 'b', '-')
l = plt.axhline(y=0)
#fig.savefig(dir_figs+str(GCM)+'_SAM_index_smoothed.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

###############################################################################
###############################################################################

import os; import shelve

filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['Depths']=my_shelf['Depths']
globals()['AMOC_max']=my_shelf['AMOC_max']
globals()['SMOC_min']=my_shelf['SMOC_min']
globals()['WS_index_rm']=my_shelf['WS_index_rm']
my_shelf.close()


AMOC_max_30S= AMOC_max[:,60] # AMOC at 30S # Max of streamfunction method
AMOC_max_30S_m=runningMeanFast(AMOC_max_30S, 10)
AMOC_max_30N= AMOC_max[:,120] # AMOC at 30N # Max of streamfunction method
AMOC_max_30N_m=runningMeanFast(AMOC_max_30N, 10)
SMOC_min_30S= SMOC_min[:,60] # SMOC at 30S # Max of streamfunction method
SMOC_min_30S_m=runningMeanFast(SMOC_min_30S, 10)


####################################################################
### AMOC/SMOC Lagged Corrolation Plots - Max/Min Stream function ###
lag_time=40

fig=plt.figure()
lag_cor(WS_index_rm[:-9],SAM_index_rm[:-9],lag_time,'b','SAM index')
lag_cor(WS_index_rm[:-9],AMOC_max_30N_m[:-9],lag_time,'y','AMOC 30N')
lag_cor(WS_index_rm[:-9],AMOC_max_30S_m[:-9],lag_time,'g','AMOC 30S')
lag_cor(WS_index_rm[:-9],SMOC_min_30S_m[:-9],lag_time,'r','SMOC 30S')
plt.legend()
plt.title('Peak WS Convection lagged correlation [AMOC=max streamfunction] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_new_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
###############################################################################
###############################################################################
Temp_allyears=np.load('Temp_allyears_GFDL-ESM2G_500yr.npy')

Temp_WS_Box_surf = Temp_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Temp_WS_Box_surf = np.nanmean(np.nanmean(Temp_WS_Box_surf,axis=1),axis=1)
Temp_WS_Box_surf_m=runningMeanFast(Temp_WS_Box_surf, 10)

Temp_WS_Box_500m = copy.deepcopy(Temp_allyears[:,:,20:35,320:]) # [40W-0W,70S-55S]
Temp_WS_Box_500m = np.nanmean( np.nanmean (Temp_WS_Box_500m, axis=3), axis=2)
Temp_WS_Box_500m = ( (Temp_WS_Box_500m[:,30]*53.04 +Temp_WS_Box_500m[:,29]*24.17 ) / (53.04+24.17) ) # Interpolating between Depth[30]=524.17 and Depth[29]=446.94 to calculate value at depth 500m
Temp_WS_Box_500m_m=runningMeanFast(Temp_WS_Box_500m, 10)

del Temp_allyears
###############################
Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')

Salinity_WS_Box_surf = Salinity_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Salinity_WS_Box_surf = np.nanmean(np.nanmean(Salinity_WS_Box_surf,axis=1),axis=1)
Salinity_WS_Box_surf_m=runningMeanFast(Salinity_WS_Box_surf, 10)

Salinity_WS_Box_500m = copy.deepcopy(Salinity_allyears[:,:,20:35,320:]) # [40W-0W,70S-55S]
Salinity_WS_Box_500m = np.nanmean( np.nanmean (Salinity_WS_Box_500m, axis=3), axis=2)
Salinity_WS_Box_500m = ( (Salinity_WS_Box_500m[:,30]*53.04 +Salinity_WS_Box_500m[:,29]*24.17 ) / (53.04+24.17) ) # Interpolating between Depth[30]=524.17 and Depth[29]=446.94 to calculate value at depth 500m
Salinity_WS_Box_500m_m=runningMeanFast(Salinity_WS_Box_500m, 10)

del Salinity_allyears
###############################
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

Density_WS_Box_surf = Density_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Density_WS_Box_surf = np.nanmean(np.nanmean(Density_WS_Box_surf,axis=1),axis=1)
Density_WS_Box_surf_m=runningMeanFast(Density_WS_Box_surf, 10)

Density_WS_Box_500m = copy.deepcopy(Density_allyears[:,:,20:35,320:]) # [40W-0W,70S-55S]
Density_WS_Box_500m = np.nanmean( np.nanmean (Density_WS_Box_500m, axis=3), axis=2)
Density_WS_Box_500m = ( (Density_WS_Box_500m[:,30]*53.04 +Density_WS_Box_500m[:,29]*24.17 ) / (53.04+24.17) ) # Interpolating between Depth[30]=524.17 and Depth[29]=446.94 to calculate value at depth 500m
Density_WS_Box_500m_m=runningMeanFast(Density_WS_Box_500m, 10)

del Density_allyears
###############################
HFDS_WS_Box=np.load('HFDS_final_GFDL-ESM2G_500yr.npy')
HFDS_WS_Box = np.nanmean(np.nanmean(HFDS_WS_Box[:,20:35,320:],axis=1),axis=1)
HFDS_WS_Box_m=runningMeanFast(HFDS_WS_Box, 10)
###############################
SIC_WS_Box=np.load('SAM_index_GFDL-ESM2G_500yr.npy')
SIC_WS_Box = np.nanmean(np.nanmean(SIC_Annual_allyears[:,20:35,320:],axis=1),axis=1)
SIC_WS_Box_m=runningMeanFast(SIC_WS_Box, 10)
###############################
SAM_index=np.load('SAM_index_GFDL-ESM2G_500yr.npy')
SAM_index_m=runningMeanFast(SAM_index, 10)
###############################
ACC_0_1000m = np.load('GFDL-ESM2G_ACC_500yr_transport_0_1000m_ACC.npy')
ACC_0_1000m = np.nansum(ACC_0_1000m[:,25:35,295],axis=1)
ACC_0_1000m_m=runningMeanFast(ACC_0_1000m, 10)

ACC_3000_4200m = np.load('GFDL-ESM2G_ACC_500yr_transport_3000_4200m_ACC.npy')
ACC_3000_4200m = np.nansum(ACC_3000_4200m[:,25:35,295],axis=1)
ACC_3000_4200m_m=runningMeanFast(ACC_3000_4200m, 10)
###############################

lag_time=40

fig=plt.figure()
lag_cor(WS_index_rm[:-9],Temp_WS_Box_surf_m[:-9],lag_time,'b','Temperature at surf')
lag_cor(WS_index_rm[:-9],Temp_WS_Box_500m_m[:-9],lag_time,'c','Temperature at 500m')
lag_cor(WS_index_rm[:-9],Density_WS_Box_surf_m[:-9],lag_time,'r','Density at surf')
lag_cor(WS_index_rm[:-9],Density_WS_Box_500m_m[:-9],lag_time,'darkorange','Density at 500m')
lag_cor(WS_index_rm[:-9],Salinity_WS_Box_surf_m[:-9],lag_time,'g','Salinity at surf')
lag_cor(WS_index_rm[:-9],Salinity_WS_Box_500m_m[:-9],lag_time,'y','Salinitye at 500m')
lag_cor(WS_index_rm[:-9],HFDS_WS_Box_m[:-9],lag_time,'m','Heat Flux into ocean')
plt.legend()
plt.title('Peak WS Convection lagged correlation with WS Box [40W-0W,70S-55S] Temp/Density/Salinity \n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_WS_Box_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
lag_cor(Temp_WS_Box_surf_m[:-9],WS_index_rm[:-9],lag_time,'y','WS Index')
lag_cor(Temp_WS_Box_surf_m[:-9],HFDS_WS_Box_m[:-9],lag_time,'m','Heat Flux into ocean')
lag_cor(Temp_WS_Box_surf_m[:-9],Salinity_WS_Box_surf_m[:-9],lag_time,'g','Salinity at surf')
lag_cor(Temp_WS_Box_surf_m[:-9],SAM_index_m[:-9],lag_time,'darkorange','SAM Index')
lag_cor(Temp_WS_Box_surf_m[:-9],SIC_WS_Box_m[:-9],lag_time,'b','Sea Ice')
plt.legend()
plt.title('Peak Surfae Temperature in WS Box [40W-0W,70S-55S] lagged correlation\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_WS_Box_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
lag_cor(WS_index_rm[:-9],HFDS_WS_Box_m[:-9],lag_time,'m','Heat Flux into ocean')
lag_cor(WS_index_rm[:-9],SAM_index_m[:-9],lag_time,'darkorange','SAM Index')
lag_cor(WS_index_rm[:-9],SIC_WS_Box_m[:-9],lag_time,'b','Sea Ice')
lag_cor(WS_index_rm[:-9],ACC_0_1000m_m[:-9],lag_time,'g','ACC (0-1000m)')
lag_cor(WS_index_rm[:-9],ACC_3000_4200m_m[:-9],lag_time,'c','ACC (3000-4200m)')
plt.legend()
plt.title('Peak WS Convection lagged correlation \n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_WS_Box_3.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################
###############################################################################
###############################################################################
def interpolate_2_reg_grid(lon, lat, data):
    ny, nx = 181, 360
    xmin, xmax = 0, 359
    ymin, ymax = -90, 90
    xi = np.linspace(xmin, xmax, nx)
    yi = np.linspace(ymin, ymax, ny)
    x_i, y_i = np.meshgrid(xi, yi)
    if np.ndim(lon)==1:
        lon,lat=np.meshgrid(lon, lat)
        
    lon_i = np.asarray(lon)
    lat_i = np.asarray(lat)
    lon_i = lon_i.flatten()
    lat_i = lat_i.flatten()
    coords=np.squeeze(np.dstack((lon_i,lat_i)))

    data=np.squeeze(data)
    if data.ndim==2:#this is for 2d regridding
        z = np.asarray(data)
        z = z.flatten()
        zi = griddata(coords, z, (x_i, y_i), method='nearest')
        return x_i, y_i, zi
    if data.ndim==3:#this is for 3d regridding
        data_3d=[]
        for d in range(len(data)):
            z = np.asarray(data[d,:,:])
            z = z.flatten()
            zi = griddata(coords, z, (x_i, y_i), method='nearest')
            data_3d.append(zi)
        return x_i, y_i, np.asarray(data_3d)
    
def calc_stream(*args):
    depths=args[0].extract_depth()
    depths_b=args[0].extract_depth_bounds()
    depths_r=depths_b[:,1]-depths_b[:,0]
    start,end = args[0].find_time(args[0].times, args[1], args[2])
    stream_final=[]
    transport_final=[]
    transport_0_1000=[]
    transport_2000_3000=[]

    surf_0,surf_1=args[0].find_depth(args[0].lvl,args[5][0],args[5][1])
    deep_0,deep_1=args[0].find_depth(args[0].lvl,args[6][0],args[6][1])

    for i in range(int((end+1-start)/12)):
        print (i, start+12*i, start+12*i+12)
        data_extracted=args[0].extract_data(args[0].variable,start+12*i,start+12*i+11)
        data=np.squeeze(data_extracted)
        data=np.mean(data, axis=0)
        print (data.shape)
        lon,lat,data_i=interpolate_2_reg_grid(args[0].x,args[0].y,data)
        data_i[data_i>1000]=np.nan
        if i==0: 
            data_depth=np.full([len(lon),len(lon[0])], np.nan)
            data_depth_ranges=np.full([len(data_i),len(data_i[0]),len(data_i[0][0])], np.nan)
        [ii,jj] = args[3]
        for k in range(len(ii)):
            if sum(~np.isnan(data_i[:,ii[k],jj[k]]))>0:
                if i==0:
                    data_depth[ii[k],jj[k]]=depths[sum(~np.isnan(data_i[:,ii[k],jj[k]]))-1]
                    for l in range(sum(~np.isnan(data_i[:,ii[k],jj[k]]))):
                        data_depth_ranges[l,ii[k],jj[k]]=depths_r[l]

        if args[7]==1:           
            mul_by_lat=data_i*(np.cos(np.deg2rad(lat))*111000)
            transport=mul_by_lat*data_depth_ranges/1000000
        else:
            print('DRAKE')
            mul_by_lat=data_i*111000
            transport=mul_by_lat*data_depth_ranges/1000000
        transport_lon=np.nansum(transport,axis=2)
        stream_final.append(transport_lon)
        print(transport.shape)
        transport_0_1000.append(np.nansum(transport[surf_0:surf_1,:,:],axis=0))
        transport_2000_3000.append(np.nansum(transport[deep_0:deep_1,:,:],axis=0))

    stream_final=np.asarray(stream_final)
    transport_0_1000=np.asarray(transport_0_1000)
    transport_2000_3000=np.asarray(transport_2000_3000)
    transport_4d_mean_0_1000=np.nanmean(transport_0_1000,axis=0)
    transport_4d_mean_2000_3000=np.nanmean(transport_2000_3000,axis=0)

    maxvals=np.nanmax(transport_4d_mean_0_1000,axis=1)
    minvals=np.nanmin(transport_4d_mean_2000_3000,axis=1)

    ind_max = np.array([np.argwhere(transport_4d_mean_0_1000 == [x]) for x in maxvals])
    ind_max=np.concatenate(ind_max).astype(None)
    ind_max=np.concatenate(ind_max).astype(None)
    ii_max=ind_max[0::2].astype(int)
    jj_max=ind_max[1::2].astype(int)
    ind_min = np.array([np.argwhere(transport_4d_mean_2000_3000 == [x]) for x in minvals])
    ind_min=np.concatenate(ind_min).astype(None)
    ind_min=np.concatenate(ind_min).astype(None)
    ii_min=ind_min[0::2].astype(int)
    jj_min=ind_min[1::2].astype(int)


    return stream_final, transport_0_1000,transport_2000_3000, ii_max,jj_max, ii_min, jj_min

def calc_SO_Mask():
    ny, nx = 181, 360
    xmin, xmax = 0, 359
    ymin, ymax = -90, 90
    xi = np.linspace(xmin, xmax, nx)
    yi = np.linspace(ymin, ymax, ny)
    lon, lat = np.meshgrid(xi, yi)
    [i,j] = np.where(lat<-32)#### NH indeces####
    return [i,j]

#######################
#######################    
mask=calc_SO_Mask()
​
t_frequency='Omon'
variable='uo'# uo = sea_water_x_velocity - units: m s-1

dir_data_in1 = ('/data5/scratch/CMIP5/CMIP5_models/ocean_physics/') # Directory to raed raw data from
dir_data_in2=(dir_data_in1+ GCM + '/piControl/mo/')

dset_uo = netcdf_read (dir_data_in2+str(variable)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM), variable) # Specifies the netCDF dataset (consisting of multiple files) from which the data will be read

data_stream, transport_0_1000,transport_3000_4200,ii_max,jj_max, ii_min, jj_min  =calc_stream(dset_uo, year_start, year_end, mask, 1, [0,1000],[3000, 4200],0)
​
np.save('GFDL-ESM2G_ACC_500yr_data_stream_ACC.npy',data_stream)
np.save('GFDL-ESM2G_ACC_500yr_transport_0_1000m_ACC.npy',transport_0_1000)
np.save('GFDL-ESM2G_ACC_500yr_transport_3000_4200m_ACC.npy',transport_3000_4200)
​​
#######these are indeces for the ACC lon and lats####
DRAKE_surface_transport=np.nansum(transport_0_1000[:,25:35,295],axis=1)
DRAKE_deep_transport=np.nansum(transport_3000_4200[:,25:35,295],axis=1)

#ACC_0_1000m = np.load('GFDL-ESM2G_ACC_500yr_transport_0_1000m_ACC.npy')
ACC_0_1000m = np.nansum(transport_0_1000[:,25:35,295],axis=1)
ACC_0_1000m_m=runningMeanFast(ACC_0_1000m, 10)

#ACC_3000_4200m = np.load('GFDL-ESM2G_ACC_500yr_transport_3000_4200m_ACC.npy')
ACC_3000_4200m = np.nansum(transport_3000_4200[:,25:35,295],axis=1)
ACC_3000_4200m_m=runningMeanFast(ACC_3000_4200m, 10)


years=np.linspace(year_start,year_end,year_end-year_start+1)
P_Var_x=years[:-9]
P_Var_y=ACC_0_1000m_m[:-9]
P_xlable='Years'
P_ylable='ACC (Sv)'
P_title='ACC at surface (0-1000m) [SV] [smoothed] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)
fig = func_plotline_1var(P_Var_x, P_Var_y, P_xlable, P_ylable, P_title, 'b', '-')
#fig.savefig(dir_figs+str(GCM)+'_ACC_surf_smoothed.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

years=np.linspace(year_start,year_end,year_end-year_start+1)
P_Var_x=years[:-9]
P_Var_y=ACC_3000_4200m_m[:-9]
P_xlable='Years'
P_ylable='ACC (Sv)'
P_title='ACC at deep ocean (3000-4200m) [SV] [smoothed] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)
fig = func_plotline_1var(P_Var_x, P_Var_y, P_xlable, P_ylable, P_title, 'b', '-')
#fig.savefig(dir_figs+str(GCM)+'_ACC_deep_smoothed.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


###############################################################################
###############################################################################

direct= '/data6/home/braf/RUNS/Riga/riga_rds/CM2Mc_Control-1900/CM2Mc_spinup_from1500_extension2/history/'
file_name='ocean_month.nc.001'

file_dire=direct+file_name

dset_t = Dataset(file_dire)
dset_t.variables

variable='thetao'


###############################################################################
###############################################################################
############      Antarctic Intermediate Waters (AAIW)       ##################
###############################################################################


Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')
Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')

import os; import shelve

filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['Depths']=my_shelf['Depths']; my_shelf.close()



P_Var_x1=P_Var_x2=Lat_regrid_1D
P_Var_y1=P_Var_y2=Depths[0:36]
P_Var_z1=np.nanmean(Density_allyears,axis=0)
for ii in range(P_Var_z1.shape[0]):
    P_Var_z1[ii,:,:][Ocean_Index!=6]=np.nan # Atlantic Ocean only
P_Var_z1=np.nanmean(P_Var_z1,axis=2)
P_Var_z1=P_Var_z1[0:36,:]-1000

P_Var_z2=np.nanmean(Salinity_allyears,axis=0)
for ii in range(P_Var_z2.shape[0]):
    P_Var_z2[ii,:,:][Ocean_Index!=6]=np.nan # Atlantic Ocean only
P_Var_z2=np.nanmean(P_Var_z2,axis=2)
P_Var_z2=P_Var_z2[0:36,:]
P_title='Potential Density(blue) and Salinity (red) Contours [Green lines: AAIW, 27.1 <Rho< 27.6]\n Atlantic - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)
P_xlable='Latitude'  ;P_ylable='Depth [m]'
fig = func_plot2Dcontour_2var(P_Var_x1, P_Var_y1, P_Var_z1, P_Var_x2, P_Var_y2, P_Var_z2, P_xlable, P_ylable, P_title, 'b', 'r', 10, 'invert_yaxis')

P_Var_z3=copy.deepcopy(P_Var_z1)
P_Var_z3[ P_Var_z3 > 27.6]=np.nan ; P_Var_z3[ P_Var_z3 < 27.1 ]=np.nan
im2=plt.contour(P_Var_x1, P_Var_y1, P_Var_z3, 30, latlon=True, colors='darkgreen')
#fig.savefig(dir_figs+str(GCM)+'_Density_Salinity_Atlantic_AAIW_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


P_Var_x1=P_Var_x2=Lat_regrid_1D
P_Var_y1=P_Var_y2=Depths[0:38]
P_Var_z1=np.nanmean(Density_allyears,axis=0)
for ii in range(P_Var_z1.shape[0]):
    P_Var_z1[ii,:,:][Ocean_Index!=2]=np.nan # Pacific Ocean only
P_Var_z1=np.nanmean(P_Var_z1,axis=2)
P_Var_z1=P_Var_z1[0:38,:]-1000

P_Var_z2=np.nanmean(Salinity_allyears,axis=0)
for ii in range(P_Var_z2.shape[0]):
    P_Var_z2[ii,:,:][Ocean_Index!=2]=np.nan # Pacific Ocean only
P_Var_z2=np.nanmean(P_Var_z2,axis=2)
P_Var_z2=P_Var_z2[0:38,:]
P_title='Potential Density(blue) and Salinity (red) Contours [Green lines: AAIW, 27.1 <Rho< 27.6]\n Pacific - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM)
P_xlable='Latitude'  ;P_ylable='Depth [m]'
fig = func_plot2Dcontour_2var(P_Var_x1, P_Var_y1, P_Var_z1, P_Var_x2, P_Var_y2, P_Var_z2, P_xlable, P_ylable, P_title, 'b', 'r', 10, 'invert_yaxis')

P_Var_z3=copy.deepcopy(P_Var_z1)
P_Var_z3[ P_Var_z3 > 27.6]=np.nan ; P_Var_z3[ P_Var_z3 < 27.1 ]=np.nan
im2=plt.contour(P_Var_x1, P_Var_y1, P_Var_z3, 30, latlon=True, colors='darkgreen')
#fig.savefig(dir_figs+str(GCM)+'_Density_Salinity_Pacific_AAIW_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


del Density_allyears
del Salinity_allyears

###############################################################################

Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

P_Lon=Lon_regrid_2D; P_Lat=Lat_regrid_2D; P_range=np.linspace(0.999,1.001,3); P_unit='(-)'; P_cmap=plt.cm.seismic; P_proj='cyl'; P_lon0=210.; P_latN=80.; P_latS=-80.


fig=plt.figure()

plt.subplot(3,2,1)
Plot_title= 'AAIW at Surface'
Plot_Var = np.nanmean(Density_allyears[:,0,:,:],axis=0)-1000
Plot_Var_AAIW = np.nanmean(Density_allyears[:,0,:,:],axis=0)-1000
Plot_Var_AAIW [Plot_Var_AAIW < 27.1] = np.nan; Plot_Var_AAIW [Plot_Var_AAIW > 27.6] = np.nan
Plot_Var_AAIW [ ~np.isnan(Plot_Var_AAIW) ] = 1

m = Basemap( projection=P_proj, lon_0=P_lon0, llcrnrlon=P_lon0-180, llcrnrlat=P_latS, urcrnrlon=P_lon0+180, urcrnrlat=P_latN)
m.drawparallels(np.arange(P_latS, P_latN+0.001, 40.),labels=[True,False,False,False], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Latitutes
m.drawmeridians(np.arange(P_lon0-180,P_lon0+180,60.),labels=[False,False,False,True], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Longitudes        
m.drawcoastlines(linewidth=1.0, linestyle='solid', color='k', antialiased=1, ax=None, zorder=None)
im=m.contourf(P_Lon, P_Lat, Plot_Var_AAIW,P_range,latlon=True, cmap=P_cmap, extend='both')
cbar = m.colorbar(im,"right", size="3%", pad="2%")
cbar.ax.tick_params(labelsize=20) 
cbar.set_label(P_unit)
plt.title(Plot_title, fontsize=18)
mng = plt.get_current_fig_manager()


plt.subplot(3,2,3)
Plot_title= 'AAIW at 100m'
Plot_Var = np.nanmean(Density_allyears[:,10,:,:],axis=0)-1000
Plot_Var_AAIW = np.nanmean(Density_allyears[:,10,:,:],axis=0)-1000
Plot_Var_AAIW [Plot_Var_AAIW < 27.1] = np.nan; Plot_Var_AAIW [Plot_Var_AAIW > 27.6] = np.nan
Plot_Var_AAIW [ ~np.isnan(Plot_Var_AAIW) ] = 1

m = Basemap( projection=P_proj, lon_0=P_lon0, llcrnrlon=P_lon0-180, llcrnrlat=P_latS, urcrnrlon=P_lon0+180, urcrnrlat=P_latN)
m.drawparallels(np.arange(P_latS, P_latN+0.001, 40.),labels=[True,False,False,False], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Latitutes
m.drawmeridians(np.arange(P_lon0-180,P_lon0+180,60.),labels=[False,False,False,True], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Longitudes        
m.drawcoastlines(linewidth=1.0, linestyle='solid', color='k', antialiased=1, ax=None, zorder=None)
im=m.contourf(P_Lon, P_Lat, Plot_Var_AAIW,P_range,latlon=True, cmap=P_cmap, extend='both')
cbar = m.colorbar(im,"right", size="3%", pad="2%")
cbar.ax.tick_params(labelsize=20) 
cbar.set_label(P_unit)
plt.title(Plot_title, fontsize=18)
mng = plt.get_current_fig_manager()


plt.subplot(3,2,5)
Plot_title= 'AAIW at 200m'
Plot_Var = np.nanmean(Density_allyears[:,20,:,:],axis=0)-1000
Plot_Var_AAIW = np.nanmean(Density_allyears[:,20,:,:],axis=0)-1000
Plot_Var_AAIW [Plot_Var_AAIW < 27.1] = np.nan; Plot_Var_AAIW [Plot_Var_AAIW > 27.6] = np.nan
Plot_Var_AAIW [ ~np.isnan(Plot_Var_AAIW) ] = 1

m = Basemap( projection=P_proj, lon_0=P_lon0, llcrnrlon=P_lon0-180, llcrnrlat=P_latS, urcrnrlon=P_lon0+180, urcrnrlat=P_latN)
m.drawparallels(np.arange(P_latS, P_latN+0.001, 40.),labels=[True,False,False,False], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Latitutes
m.drawmeridians(np.arange(P_lon0-180,P_lon0+180,60.),labels=[False,False,False,True], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Longitudes        
m.drawcoastlines(linewidth=1.0, linestyle='solid', color='k', antialiased=1, ax=None, zorder=None)
im=m.contourf(P_Lon, P_Lat, Plot_Var_AAIW,P_range,latlon=True, cmap=P_cmap, extend='both')
cbar = m.colorbar(im,"right", size="3%", pad="2%")
cbar.ax.tick_params(labelsize=20) 
cbar.set_label(P_unit)
plt.title(Plot_title, fontsize=18)
mng = plt.get_current_fig_manager()


plt.subplot(3,2,2)
Plot_title= 'AAIW at 500m'
Plot_Var = np.nanmean(Density_allyears[:,30,:,:],axis=0)-1000
Plot_Var_AAIW = np.nanmean(Density_allyears[:,30,:,:],axis=0)-1000
Plot_Var_AAIW [Plot_Var_AAIW < 27.1] = np.nan; Plot_Var_AAIW [Plot_Var_AAIW > 27.6] = np.nan
Plot_Var_AAIW [ ~np.isnan(Plot_Var_AAIW) ] = 1

m = Basemap( projection=P_proj, lon_0=P_lon0, llcrnrlon=P_lon0-180, llcrnrlat=P_latS, urcrnrlon=P_lon0+180, urcrnrlat=P_latN)
m.drawparallels(np.arange(P_latS, P_latN+0.001, 40.),labels=[True,False,False,False], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Latitutes
m.drawmeridians(np.arange(P_lon0-180,P_lon0+180,60.),labels=[False,False,False,True], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Longitudes        
m.drawcoastlines(linewidth=1.0, linestyle='solid', color='k', antialiased=1, ax=None, zorder=None)
im=m.contourf(P_Lon, P_Lat, Plot_Var_AAIW,P_range,latlon=True, cmap=P_cmap, extend='both')
cbar = m.colorbar(im,"right", size="3%", pad="2%")
cbar.ax.tick_params(labelsize=20) 
cbar.set_label(P_unit)
plt.title(Plot_title, fontsize=18)
mng = plt.get_current_fig_manager()


plt.subplot(3,2,4)
Plot_title= 'AAIW at 1000m'
Plot_Var = np.nanmean(Density_allyears[:,34,:,:],axis=0)-1000
Plot_Var_AAIW = np.nanmean(Density_allyears[:,34,:,:],axis=0)-1000
Plot_Var_AAIW [Plot_Var_AAIW < 27.1] = np.nan; Plot_Var_AAIW [Plot_Var_AAIW > 27.6] = np.nan
Plot_Var_AAIW [ ~np.isnan(Plot_Var_AAIW) ] = 1

m = Basemap( projection=P_proj, lon_0=P_lon0, llcrnrlon=P_lon0-180, llcrnrlat=P_latS, urcrnrlon=P_lon0+180, urcrnrlat=P_latN)
m.drawparallels(np.arange(P_latS, P_latN+0.001, 40.),labels=[True,False,False,False], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Latitutes
m.drawmeridians(np.arange(P_lon0-180,P_lon0+180,60.),labels=[False,False,False,True], linewidth=0.01, color='k', fontsize=20) # labels = [left,right,top,bottom] # Longitudes        
m.drawcoastlines(linewidth=1.0, linestyle='solid', color='k', antialiased=1, ax=None, zorder=None)
im=m.contourf(P_Lon, P_Lat, Plot_Var_AAIW,P_range,latlon=True, cmap=P_cmap, extend='both')
cbar = m.colorbar(im,"right", size="3%", pad="2%")
cbar.ax.tick_params(labelsize=20) 
cbar.set_label(P_unit)
plt.title(Plot_title, fontsize=18)
mng = plt.get_current_fig_manager()

plt.suptitle('AAIW [27.1 <Rho< 27.6] at various depths - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM) , fontsize=18)
mng.window.showMaximized() # Maximizes the plot window to save figures in full
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.2, hspace=0.4)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_Depths_Global_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


##################################################################   
lat_n=Lat_regrid_2D.shape[0] # Number of Lat elements in the data
lon_n=Lon_regrid_2D.shape[1] # Number of Lon elements in the data
earth_R = 6378 # Earth Radius - Unit is kilometer (km)
GridCell_Areas = np.zeros ((lat_n, lon_n )) # A = 2*pi*R^2 |sin(lat1)-sin(lat2)| |lon1-lon2|/360 = (pi/180)R^2 |lon1-lon2| |sin(lat1)-sin(lat2)| 
for ii in range(1,lat_n-1):
    for jj in range(1,lon_n-1):
        GridCell_Areas [ii,jj] = math.fabs( (earth_R**2) * (math.pi/180) * np.absolute( (Lon_regrid_2D[ii,jj-1]+Lon_regrid_2D[ii,jj])/2  -  (Lon_regrid_2D[ii,jj]+Lon_regrid_2D[ii,jj+1])/2 )  * np.absolute( math.sin(math.radians( ( Lat_regrid_2D[ii-1,jj]+Lat_regrid_2D[ii,jj])/2 )) - math.sin(math.radians( Lat_regrid_2D[ii,jj]+Lat_regrid_2D[ii+1,jj])/2  )) )                  
for ii in range(1,lat_n-1):
    for jj in range(2,lon_n-2):
        if GridCell_Areas [ii,jj] > GridCell_Areas [ii,jj-1]*3:
            GridCell_Areas [ii,jj]=GridCell_Areas [ii,jj-1]
        if GridCell_Areas [ii,jj] > GridCell_Areas [ii,jj+1]*3:
            GridCell_Areas [ii,jj]=GridCell_Areas [ii,jj+1]
GridCell_Areas[0,:]=GridCell_Areas[1,:]; GridCell_Areas[-1,:]=GridCell_Areas[-2,:]
GridCell_Areas[:,0]=GridCell_Areas[:,1]; GridCell_Areas[:,-1]=GridCell_Areas[:,-2] 
# GridCell_Areas unit = km2

GridCell_volume = np.full([Depths.shape[0],Lat_regrid_2D.shape[0],Lat_regrid_2D.shape[1]],np.nan)
for l in range(GridCell_volume.shape[0]):
    GridCell_volume[l,:,:] = GridCell_Areas * Depths[l]

##################################################################   
AAIW_volume=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
    AAIW_volume[t]= np.nansum(np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=0),axis=0)

AAIW_volume_atl=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    AAIW_volume_atl[t]= np.nansum(np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=0),axis=0)

AAIW_volume_pacfc=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    AAIW_volume_pacfc[t]= np.nansum(np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=0),axis=0)



fig=plt.figure()
years=np.linspace(year_start,year_end,year_end-year_start+1)
plt.plot(years, AAIW_volume/1e9,'b')
plt.xlabel('Years', fontsize=18)
plt.ylabel('AAIW Volume (km3)', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Global AAIW Volume [AAIW: 27.1 <Rho< 27.6] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_AAIW_volume_timeseries_global_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
years=np.linspace(year_start,year_end,year_end-year_start+1)
plt.plot(years, AAIW_volume_atl/1e9,'b')
plt.xlabel('Years', fontsize=18)
plt.ylabel('AAIW Volume (km3)', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Atlantic Ocean AAIW Volume [AAIW: 27.1 <Rho< 27.6] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_AAIW_volume_timeseries_atlantic_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
years=np.linspace(year_start,year_end,year_end-year_start+1)
plt.plot(years, AAIW_volume_pacfc/1e9,'b')
plt.xlabel('Years', fontsize=18)
plt.ylabel('AAIW Volume (km3)', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Pacific Ocean AAIW Volume [AAIW: 27.1 <Rho< 27.6] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_AAIW_volume_timeseries_pacific_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

################################################################
AAIW_volume_rm = runningMeanFast(AAIW_volume, 10)
AAIW_volume_atl_rm = runningMeanFast(AAIW_volume_atl, 10)
AAIW_volume_pacfc_rm = runningMeanFast(AAIW_volume_pacfc, 10)
###############################################################################
import os; import shelve

filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['Depths']=my_shelf['Depths']
globals()['AMOC_max']=my_shelf['AMOC_max']
globals()['SMOC_min']=my_shelf['SMOC_min']
globals()['WS_index']=my_shelf['WS_index']
globals()['LAB_index']=my_shelf['LAB_index']; my_shelf.close()
my_shelf.close()


AMOC_max_30S= AMOC_max[:,60] # AMOC at 30S # Max of streamfunction method
AMOC_max_30S_m=runningMeanFast(AMOC_max_30S, 10)
AMOC_max_30N= AMOC_max[:,120] # AMOC at 30N # Max of streamfunction method
AMOC_max_30N_m=runningMeanFast(AMOC_max_30N, 10)
SMOC_min_30S= SMOC_min[:,60] # SMOC at 30S # Max of streamfunction method
SMOC_min_30S_m=runningMeanFast(SMOC_min_30S, 10)


WS_index_rm = copy.deepcopy(WS_index)
#multiplying by -1 as we have convection when temperature is low(opposite to the index which we calculate)
WS_index_rm=runningMeanFast(WS_index_rm, 10)
WS_index_rm=WS_index_rm*(-1)

LAB_index_rm = copy.deepcopy(LAB_index)
LAB_index_rm=runningMeanFast(LAB_index_rm, 10)



lag_time=40

fig=plt.figure()

lag_cor(WS_index_rm[:-9],AAIW_volume_rm[:-9],lag_time,'b','AAIW Volume - Global')
lag_cor(WS_index_rm[:-9],AAIW_volume_pacfc_rm[:-9],lag_time,'r','AAIW Volume - Pacific')
lag_cor(WS_index_rm[:-9],AAIW_volume_atl_rm[:-9],lag_time,'g','AAIW Volume - Atlantic')
plt.legend()
plt.title('Peak WS convection with AAIW volume lagged correlation - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_AAIW_vol_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
lag_cor(AAIW_volume_atl_rm[:-9], LAB_index_rm[:-9], lag_time, 'y', 'LAB index')
lag_cor(AAIW_volume_atl_rm[:-9], WS_index_rm[:-9], lag_time, 'm', 'WS index')
lag_cor(AAIW_volume_atl_rm[:-9], AMOC_max_50N_m[:-9], lag_time, 'b', 'AMOC max 50N')
lag_cor(AAIW_volume_atl_rm[:-9], AMOC_max_30N_m[:-9], lag_time, 'g', 'AMOC max 30N')
lag_cor(AAIW_volume_atl_rm[:-9], AMOC_max_30S_m[:-9], lag_time, 'darkorange', 'AMOC max 30S')
lag_cor(AAIW_volume_atl_rm[:-9], AMOC_max_50S_m[:-9], lag_time, 'r', 'AMOC max 50S')
plt.legend()
plt.title('Peak Atlantic Ocean AAIW volume lagged correlation - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_AAIW_atlantic_vol_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
lag_cor(AAIW_volume_rm[:-9], LAB_index_rm[:-9], lag_time, 'y', 'LAB index')
lag_cor(AAIW_volume_rm[:-9], WS_index_rm[:-9], lag_time, 'm', 'WS index')
lag_cor(AAIW_volume_rm[:-9], AMOC_max_50N_m[:-9], lag_time, 'b', 'AMOC max 50N')
lag_cor(AAIW_volume_rm[:-9], AMOC_max_30N_m[:-9], lag_time, 'g', 'AMOC max 30N')
lag_cor(AAIW_volume_rm[:-9], AMOC_max_30S_m[:-9], lag_time, 'darkorange', 'AMOC max 30S')
lag_cor(AAIW_volume_rm[:-9], AMOC_max_50S_m[:-9], lag_time, 'r', 'AMOC max 50S')
plt.legend()
plt.title('Peak Global AAIW volume lagged correlation - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_AAIW_global_vol_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


##################################################################   
AAIW_volume_30S=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
    AAIW_volume_30S[t]= np.nansum(np.nansum(GridCell_volume_yr[:,60,:],axis=0),axis=0)

AAIW_volume_atl_30S=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    AAIW_volume_atl_30S[t]= np.nansum(np.nansum(GridCell_volume_yr[:,60,:],axis=0),axis=0)

AAIW_volume_pacfc_30S=np.full([Density_allyears.shape[0]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    AAIW_volume_pacfc_30S[t]= np.nansum(np.nansum(GridCell_volume_yr[:,60,:],axis=0),axis=0)


AAIW_volume_30S_rm = runningMeanFast(AAIW_volume_30S, 10)
AAIW_volume_atl_30S_rm = runningMeanFast(AAIW_volume_atl_30S, 10)
AAIW_volume_pacfc_30S_rm = runningMeanFast(AAIW_volume_pacfc_30S, 10)

lag_time=40

fig=plt.figure()
lag_cor(WS_index_rm[:-9],AAIW_volume_30S_rm[:-9],lag_time,'b','AAIW Volume 30S - Global')
lag_cor(WS_index_rm[:-9],AAIW_volume_pacfc_30S_rm[:-9],lag_time,'r','AAIW Volume 30S - Pacific')
lag_cor(WS_index_rm[:-9],AAIW_volume_atl_30S_rm[:-9],lag_time,'g','AAIW Volume 30S - Atlantic')
plt.legend()
plt.title('Peak WS convection with AAIW volume at 30S lagged correlation\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 15})
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_AAIW_vol_30S_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
lag_cor(WS_index_rm[:-9],AAIW_volume_30S_rm[:-9],lag_time,'b','AAIW Vol. 30S - Global')
lag_cor(WS_index_rm[:-9],AAIW_volume_pacfc_30S_rm[:-9],lag_time,'r','AAIW Vol. 30S - Pacific')
lag_cor(WS_index_rm[:-9],AAIW_volume_atl_30S_rm[:-9],lag_time,'g','AAIW Vol. 30S - Atlantic')
lag_cor(WS_index_rm[:-9], AMOC_max_30S_m[:-9], lag_time, 'darkorange', 'AMOC max 30S')
lag_cor(WS_index_rm[:-9], SMOC_min_30S_m[:-9], lag_time, 'm', 'SMOC min 30S')

plt.legend()
plt.title('Peak WS convection with AAIW volume at 30S lagged correlation\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
plt.xlabel('Years lag', fontsize=18)
plt.ylabel('Correlation coefficient', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.legend(shadow=True, prop={'size': 10})
mng = plt.get_current_fig_manager()
plt.ylim(-1,1)
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_WSconvection_AAIW_vol_30S_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


##################################################################   
AAIW_volume_atl_lat=np.full([Density_allyears.shape[0], Density_allyears.shape[2]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    AAIW_volume_atl_lat[t,:]= np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=1)

AAIW_volume_pacfc_lat=np.full([Density_allyears.shape[0], Density_allyears.shape[2]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    AAIW_volume_pacfc_lat[t,:]= np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=1)


fig=plt.figure()
Plot_range=np.linspace(-4,4,81)

Var_plot = AAIW_volume_atl_lat
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of AAIW volume [Normalized, (var-mean)/std] in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = AAIW_volume_pacfc_lat
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of AAIW volume [Normalized, (var-mean)/std] in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_volume_time_lat_norm_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

##################################################################   
AAIW_volume_atl_east_lat=np.full([Density_allyears.shape[0], Density_allyears.shape[2]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
        GridCell_volume_yr[l,:,200:335]=np.nan
    AAIW_volume_atl_east_lat[t,:]= np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=1)

AAIW_volume_atl_west_lat=np.full([Density_allyears.shape[0], Density_allyears.shape[2]],np.nan)
for t in range(Density_allyears.shape[0]):
    GridCell_volume_yr = copy.deepcopy(GridCell_volume)
    for l in range(Density_allyears.shape[1]):
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        GridCell_volume_yr[l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        GridCell_volume_yr[l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
        GridCell_volume_yr[l,:,0:100]=np.nan; GridCell_volume_yr[l,:,335:]=np.nan
    AAIW_volume_atl_west_lat[t,:]= np.nansum(np.nansum(GridCell_volume_yr,axis=0),axis=1)


fig=plt.figure()
Plot_range=np.linspace(-4,4,81)

Var_plot = AAIW_volume_atl_east_lat
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of AAIW volume [Normalized, (var-mean)/std] in East Atlantic Ocean Salinity [east of 25W] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()


Var_plot = AAIW_volume_atl_west_lat
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of AAIW volume [Normalized, (var-mean)/std] in West Atlantic Ocean Salinity [west of 25W] - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

mng.window.showMaximized() # Maximizes the plot window to save figures in full

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_volume_time_lat_norm_Atl_E_W_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

###############################################################################

lag_time=40

fig=plt.figure()
data=[]
for i in range(len(AAIW_volume_atl_lat[0][20:161])):
    stream=runningMeanFast(AAIW_volume_atl_lat[:,i+35], 10)
    r=lag_cor_data(WS_index_rm,stream,lag_time)
    data.append(r)
data=np.asarray(data)
x=np.linspace(-lag_time,lag_time+1, 2*lag_time)
y=np.linspace(-70,70+1, 2*70+1)

x,y=np.meshgrid(x,y)
cmap_limit=np.nanmax(np.abs(data))
levels=np.linspace(-cmap_limit,cmap_limit,200)
im=plt.contourf(x,y,data,levels,latlon=True, cmap=plt.cm.jet)
cb = plt.colorbar(im)
plt.xlabel('Year lag', fontsize=18)
plt.ylabel('Latitude', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('WS peak convection vs. AAIW volume in Atlantic Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_convec_AAIW_atlantic_WS_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


fig=plt.figure()
data=[]
for i in range(len(AAIW_volume_pacfc_lat[0][20:161])):
    stream=runningMeanFast(AAIW_volume_pacfc_lat[:,i+35], 10)
    r=lag_cor_data(WS_index_rm,stream,lag_time)
    data.append(r)
data=np.asarray(data)
x=np.linspace(-lag_time,lag_time+1, 2*lag_time)
y=np.linspace(-70,70+1, 2*70+1)

x,y=np.meshgrid(x,y)
cmap_limit=np.nanmax(np.abs(data))
levels=np.linspace(-cmap_limit,cmap_limit,200)
im=plt.contourf(x,y,data,levels,latlon=True, cmap=plt.cm.jet)
cb = plt.colorbar(im)
plt.xlabel('Year lag', fontsize=18)
plt.ylabel('Latitude', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('WS peak convection vs. AAIW volume in Pacific Ocean - '+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
#fig.savefig(dir_figs+str(GCM)+'_lagcor_convec_AAIW_pacific_WS_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


################################################################################
# Propagation  of Salinity for AAIW (27.1 <Rho< 27.6)[Normalized,  (var-mean)/std] in Atlantic and Pacific Oceans

## Averaging Method #1:
# salinity gridcellswhere corresponding density values are outside the 1027.1-1027.6  bound  are excluded from all layers and all years, 
# then salinity values averaged over each ocean basin for each year and plotted

Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

for t in range(Salinity_allyears.shape[0]):
    for l in range(Salinity_allyears.shape[1]):
        Salinity_allyears[t,l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        Salinity_allyears[t,l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        
Salinity_time_lat_atl_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    Salinity_time_lat_atl_AAIW.append(np.nanmean(data_i,axis=1))    
Salinity_time_lat_atl_AAIW = np.asarray(Salinity_time_lat_atl_AAIW)

Salinity_time_lat_pacfc_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=2 ] = np.nan # Keep Pacific only
    Salinity_time_lat_pacfc_AAIW.append(np.nanmean(data_i,axis=1))    
Salinity_time_lat_pacfc_AAIW = np.asarray(Salinity_time_lat_pacfc_AAIW)

del Salinity_allyears
del Density_allyears


fig=plt.figure()
Plot_range=np.linspace(-4,4,81)

Var_plot = Salinity_time_lat_atl_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of Atlantic Ocean Salinity for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

Var_plot = Salinity_time_lat_pacfc_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of Pacific Ocean Salinity for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_Salinity_time_lat_norm_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


## Averaging Method #2:
# salinity and density values are first averaged over each ocean basin and THEN salinity latitude bounds
# where corresponding averaged density values are outside the 1027.1-1027.6  bound are excluded from all latitudes and all years,
# and plotted  –Top 100m  layers are also excluded

Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

for t in range(Salinity_allyears.shape[0]):
    for l in range(Salinity_allyears.shape[1]):
        Salinity_allyears[t,l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
        Density_allyears[t,l,:,:][ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
        
Salinity_time_lat_atl_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=2) # Averaging over longitude
    data_i_2=Density_allyears[t,:,:,:]
    data_i_2 = np.nanmean(data_i_2, axis=2) # Averaging over longitude
    data_i[data_i_2 > 1027.6] = np.nan
    data_i[data_i_2 < 1027.1] = np.nan
   
    Salinity_time_lat_atl_AAIW.append(np.nanmean(data_i,axis=0))    
Salinity_time_lat_atl_AAIW = np.asarray(Salinity_time_lat_atl_AAIW)

del Salinity_allyears
del Density_allyears

Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

for t in range(Salinity_allyears.shape[0]):
    for l in range(Salinity_allyears.shape[1]):
        Salinity_allyears[t,l,:,:][ Ocean_Index!=2 ] = np.nan # Keep Pacific only
        Density_allyears[t,l,:,:][ Ocean_Index!=2 ] = np.nan # Keep Pacific only
        
Salinity_time_lat_pacfc_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=2) # Averaging over longitude
    data_i_2=Density_allyears[t,:,:,:]
    data_i_2 = np.nanmean(data_i_2, axis=2) # Averaging over longitude
    data_i[data_i_2 > 1027.6] = np.nan
    data_i[data_i_2 < 1027.1] = np.nan
   
    Salinity_time_lat_pacfc_AAIW.append(np.nanmean(data_i,axis=0))    
Salinity_time_lat_pacfc_AAIW = np.asarray(Salinity_time_lat_pacfc_AAIW)

del Salinity_allyears
del Density_allyears


fig=plt.figure()
Plot_range=np.linspace(-4,4,81)

Var_plot = Salinity_time_lat_atl_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of Atlantic Ocean Salinity for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

Var_plot = Salinity_time_lat_pacfc_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of Pacific Ocean Salinity for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_Salinity_time_lat_norm_2.png', format='png', dpi=300, transparent=True, bbox_inches='tight')


################################################################################
# Propagation  of Salinity for AAIW (27.1 <Rho< 27.6)[Normalized,  (var-mean)/std] in East and West Atlantic

## Averaging Method #1:
# salinity gridcellswhere corresponding density values are outside the 1027.1-1027.6  bound  are excluded from all layers and all years, 
# then salinity values averaged over each ocean basin for each year and plotted

Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

for t in range(Salinity_allyears.shape[0]):
    for l in range(Salinity_allyears.shape[1]):
        Salinity_allyears[t,l,:,:][Density_allyears[t,l,:,:] > 1027.6] = np.nan
        Salinity_allyears[t,l,:,:][Density_allyears[t,l,:,:] < 1027.1] = np.nan
        
Salinity_time_lat_atl_east_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    data_i[:,200:335]=np.nan
    Salinity_time_lat_atl_east_AAIW.append(np.nanmean(data_i,axis=1))    
Salinity_time_lat_atl_east_AAIW = np.asarray(Salinity_time_lat_atl_east_AAIW)

Salinity_time_lat_atl_west_AAIW=[] 
for t in range(Salinity_allyears.shape[0]):
    data_i=Salinity_allyears[t,:,:,:]
    data_i = np.nanmean(data_i, axis=0)
    data_i[ Ocean_Index!=6 ] = np.nan # Keep Atlantic only
    data_i[:,0:100]=np.nan; data_i[:,335:]=np.nan
    Salinity_time_lat_atl_west_AAIW.append(np.nanmean(data_i,axis=1))    
Salinity_time_lat_atl_west_AAIW = np.asarray(Salinity_time_lat_atl_west_AAIW)

del Salinity_allyears
del Density_allyears


fig=plt.figure()
Plot_range=np.linspace(-4,4,81)

Var_plot = Salinity_time_lat_atl_east_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,1)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of East Atlantic Ocean Salinity [east of 25W] for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()

Var_plot = Salinity_time_lat_atl_west_AAIW
aa = np.nanmean(Var_plot,axis=0)
cc = np.std(Var_plot,axis=0)
bb = copy.deepcopy(Var_plot)
dd = copy.deepcopy(Var_plot)
for ii in range(Var_plot.shape[0]):
    bb[ii,:]=aa
    dd[ii,:]=cc

plt.subplot(2,1,2)
years=np.linspace(year_start,year_end,year_end-year_start+1)
im = plt.contourf(years, Lat_regrid_1D[12:170], np.transpose( (Var_plot[:,12:170]-bb[:,12:170])/dd[:,12:170] ), Plot_range, cmap=plt.cm.seismic, extend='both')
#plt.gca().invert_yaxis()
plt.xlabel('Years', fontsize=18)
plt.ylabel('Latitude N', fontsize=18)
plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
plt.title('Propagation of West Atlantic Ocean Salinity [west of 25W] for AAIW [27.1 <Rho< 27.6] [Normalized, (var-mean)/std]\n'+str(year_start)+'-'+str(year_end)+' - '+str(GCM), fontsize=18)
cbar = plt.colorbar(im, extend='max')
#cbar = m.colorbar(im,"right", size="3%", pad="2%", extend='max') # extend='both' will extend the colorbar in both sides (upper side and down side)
cbar.set_label('(-)')
mng = plt.get_current_fig_manager()
mng.window.showMaximized() # Maximizes the plot window to save figures in full
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
#fig.savefig(dir_figs+str(GCM)+'_AAIW_Salinity_time_lat_norm_atl_E_W_1.png', format='png', dpi=300, transparent=True, bbox_inches='tight')

###############################################################################
###############################################################################
##############                     The Table                     ##############

### Southern Ocean Global Indices

# Average (500yr) SST south of 55S
SST_SO_ave=np.load('Temp_allyears_GFDL-ESM2G_500yr.npy')
SST_SO_ave = SST_SO_ave[:,0,0:36,:]
SST_SO_ave = np.nanmean(np.nanmean(np.nanmean(SST_SO_ave,axis=2),axis=1),axis=0) - 273.15


# Average (500yr) Salinity SSS south of 55S
SSS_SO_ave=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')
SSS_SO_ave = SSS_SO_ave[:,0,0:36,:]
SSS_SO_ave = np.nanmean(np.nanmean(np.nanmean(SSS_SO_ave,axis=2),axis=1),axis=0)


## Average (500yr) Sept sea ice extent (10^6 km2) in the SO
t_frequency='OImon'
variable_sic='sic' # Sea Ice Concentration
month_no=9 # For September
dir_data_in1 = ('/data5/scratch/CMIP5/CMIP5_models/ocean_physics/') # Directory to raed raw data from
dir_data_in2=(dir_data_in1+ GCM + '/piControl/mo/')

dset_sic = netcdf_read (dir_data_in2+str(variable_sic)+'_'+str(t_frequency)+'_'+str(GCM)+'*12.nc',str(GCM), variable_sic) # Specifies the netCDF dataset (consisting of multiple files) from which the data will be read
start_date_i,end_date_i = dset_sic.find_time(dset_sic.times, year_start, year_end)

SIC_Sept_allyears=np.full([int((end_date_i+1-start_date_i)/12),Lat_regrid_2D.shape[0],Lon_regrid_2D.shape[1]], np.nan)
for t in range(int((end_date_i+1-start_date_i)/12)):
    if t%20==0: # Just showing every 10 year
        print('SIC calc - Year: ', year_start+t)
    data_sic_extracted=dset_sic.extract_data(dset_sic.variable, start_date_i+12*t+month_no-1,start_date_i+12*t+month_no-1)
    
    data_sic_extracted=np.nanmean(data_sic_extracted, axis=0)  
    data_sic_extracted=np.squeeze(data_sic_extracted)
    data_i = func_regrid(data_sic_extracted, dset_sic.y, dset_sic.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>1e19]=np.nan
    
    SIC_Sept_allyears[t,:,:]=data_i

SIC_Sept_allyears[SIC_Sept_allyears < 15 ] = np.nan # Sea Ice Extent is defined as sea ice concentration > 15%
SIC_Sept_allyears[:,90:,:] = np.nan # only keeping Southern Hemisphere data


lat_n=Lat_regrid_2D.shape[0] # Number of Lat elements in the data
lon_n=Lon_regrid_2D.shape[1] # Number of Lon elements in the data
earth_R = 6378 # Earth Radius - Unit is kilometer (km)
GridCell_Areas = np.zeros ((lat_n, lon_n )) # A = 2*pi*R^2 |sin(lat1)-sin(lat2)| |lon1-lon2|/360 = (pi/180)R^2 |lon1-lon2| |sin(lat1)-sin(lat2)| 
for ii in range(1,lat_n-1):
    for jj in range(1,lon_n-1):
        GridCell_Areas [ii,jj] = math.fabs( (earth_R**2) * (math.pi/180) * np.absolute( (Lon_regrid_2D[ii,jj-1]+Lon_regrid_2D[ii,jj])/2  -  (Lon_regrid_2D[ii,jj]+Lon_regrid_2D[ii,jj+1])/2 )  * np.absolute( math.sin(math.radians( ( Lat_regrid_2D[ii-1,jj]+Lat_regrid_2D[ii,jj])/2 )) - math.sin(math.radians( Lat_regrid_2D[ii,jj]+Lat_regrid_2D[ii+1,jj])/2  )) )                  
for ii in range(1,lat_n-1):
    for jj in range(2,lon_n-2):
        if GridCell_Areas [ii,jj] > GridCell_Areas [ii,jj-1]*3:
            GridCell_Areas [ii,jj]=GridCell_Areas [ii,jj-1]
        if GridCell_Areas [ii,jj] > GridCell_Areas [ii,jj+1]*3:
            GridCell_Areas [ii,jj]=GridCell_Areas [ii,jj+1]
GridCell_Areas[0,:]=GridCell_Areas[1,:]; GridCell_Areas[-1,:]=GridCell_Areas[-2,:]
GridCell_Areas[:,0]=GridCell_Areas[:,1]; GridCell_Areas[:,-1]=GridCell_Areas[:,-2] 
# GridCell_Areas unit = km2

SIC_Sept_Area_allyears = np.full([SIC_Sept_allyears.shape[0],SIC_Sept_allyears.shape[1],SIC_Sept_allyears.shape[2]],np.nan)
for ii in range(SIC_Sept_Area_allyears.shape[0]):
    SIC_Sept_Area_allyears[ii,:,:] = GridCell_Areas

SIC_Sept_Area_allyears [ np.isnan(SIC_Sept_allyears) ] = np.nan
SIC_Sept_Area_ave = np.nanmean( np.nansum(np.nansum(SIC_Sept_Area_allyears,axis=2),axis=1) , axis=0)


# Average (500yr) February sea ice extent (10^6 km2) in the SO
month_no=2 # For February

SIC_Feb_allyears=np.full([int((end_date_i+1-start_date_i)/12),Lat_regrid_2D.shape[0],Lon_regrid_2D.shape[1]], np.nan)
for t in range(int((end_date_i+1-start_date_i)/12)):
    if t%20==0: # Just showing every 10 year
        print('SIC calc - Year: ', year_start+t)
    data_sic_extracted=dset_sic.extract_data(dset_sic.variable, start_date_i+12*t+month_no-1,start_date_i+12*t+month_no-1)
    
    data_sic_extracted=np.nanmean(data_sic_extracted, axis=0)  
    data_sic_extracted=np.squeeze(data_sic_extracted)
    data_i = func_regrid(data_sic_extracted, dset_sic.y, dset_sic.x, Lat_regrid_2D, Lon_regrid_2D)
    data_i[data_i>1e19]=np.nan
    
    SIC_Feb_allyears[t,:,:]=data_i

SIC_Feb_allyears[SIC_Feb_allyears < 15 ] = np.nan # Sea Ice Extent is defined as sea ice concentration > 15%
SIC_Feb_allyears[:,90:,:] = np.nan # only keeping Southern Hemisphere data

SIC_Feb_Area_allyears = np.full([SIC_Feb_allyears.shape[0],SIC_Feb_allyears.shape[1],SIC_Feb_allyears.shape[2]],np.nan)
for ii in range(SIC_Feb_Area_allyears.shape[0]):
    SIC_Feb_Area_allyears[ii,:,:] = GridCell_Areas

SIC_Feb_Area_allyears [ np.isnan(SIC_Feb_allyears) ] = np.nan
SIC_Feb_Area_ave = np.nanmean( np.nansum(np.nansum(SIC_Feb_Area_allyears,axis=2),axis=1) , axis=0)


# WS surface and 2000m density [40W-0W,70S-55S] 
Density_allyears=np.load('Density_allyears_GFDL-ESM2G_500yr.npy')

Density_WS_Box_surf_ave = Density_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Density_WS_Box_surf_ave = np.nanmean(np.nanmean(np.nanmean(Density_WS_Box_surf_ave,axis=2),axis=1),axis=0)

Density_WS_Box_2000m_ave = Density_allyears[:,39,20:35,320:] # WS [40W-0W,70S-55S]
Density_WS_Box_2000m_ave = np.nanmean(np.nanmean(np.nanmean(Density_WS_Box_2000m_ave,axis=2),axis=1),axis=0)

del Density_allyears

#############################
import os; import shelve
filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['WS_index_norm_rm']=my_shelf['WS_index_norm_rm']
my_shelf.close()

R_ii_WS = np.where(WS_index_norm_rm <-0.5)
R_jj_WS = np.where(WS_index_norm_rm >0.5)

# WS surface salinity and temperatures, 500 year avg; nonconv average, conv average [40W-0W,70S-55S] 
Temp_allyears=np.load('Temp_allyears_GFDL-ESM2G_500yr.npy')

Temp_WS_Box_surf_ave = Temp_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Temp_WS_Box_surf_ave = np.nanmean(np.nanmean(np.nanmean(Temp_WS_Box_surf_ave,axis=2),axis=1),axis=0) - 273.15

Temp_WS_Box_surf_ave_c = Temp_allyears[R_ii_WS][:,0,20:35,320:] # WS [40W-0W,70S-55S]
Temp_WS_Box_surf_ave_c = np.nanmean(np.nanmean(np.nanmean(Temp_WS_Box_surf_ave_c,axis=2),axis=1),axis=0) - 273.15

Temp_WS_Box_surf_ave_n = Temp_allyears[R_jj_WS][:,0,20:35,320:] # WS [40W-0W,70S-55S]
Temp_WS_Box_surf_ave_n = np.nanmean(np.nanmean(np.nanmean(Temp_WS_Box_surf_ave_n,axis=2),axis=1),axis=0) - 273.15


Salinity_allyears=np.load('Salinity_allyears_GFDL-ESM2G_500yr.npy')

Salinity_WS_Box_surf_ave = Salinity_allyears[:,0,20:35,320:] # WS [40W-0W,70S-55S]
Salinity_WS_Box_surf_ave = np.nanmean(np.nanmean(np.nanmean(Salinity_WS_Box_surf_ave,axis=2),axis=1),axis=0)

Salinity_WS_Box_surf_ave_c = Salinity_allyears[R_ii_WS][:,0,20:35,320:] # WS [40W-0W,70S-55S]
Salinity_WS_Box_surf_ave_c = np.nanmean(np.nanmean(np.nanmean(Salinity_WS_Box_surf_ave_c,axis=2),axis=1),axis=0)

Salinity_WS_Box_surf_ave_n = Salinity_allyears[R_jj_WS][:,0,20:35,320:] # WS [40W-0W,70S-55S]
Salinity_WS_Box_surf_ave_n = np.nanmean(np.nanmean(np.nanmean(Salinity_WS_Box_surf_ave_n,axis=2),axis=1),axis=0)

del Temp_allyears
del Salinity_allyears

# SAM Index: 500 year avg; nonconv average, conv average
SAM_index=np.load('SAM_index_GFDL-ESM2G_500yr.npy')

np.nanmean(SAM_index) # All years average

np.nanmean(SAM_index[R_ii_WS]) # Convective years averge

np.nanmean(SAM_index[R_jj_WS]) # Non-Convective years averge

# September (500yr) Wedell Sea Convection area
filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['Conv_area_timeseries_WS']=my_shelf['Conv_area_timeseries_WS']
my_shelf.close()

np.nanmean(Conv_area_timeseries_WS) * 1e-6 # KM2  # All years average

np.nanmean(Conv_area_timeseries_WS[R_ii_WS]) * 1e-6 # KM2 # Convective years averge

np.nanmean(Conv_area_timeseries_WS[R_jj_WS]) * 1e-6 # KM2 # Non-Convective years averge

# SMOC (Atlantic) streamfunction (30S)
filename_out = (dir_pwd + '/Results/AllResults_'+GCM+'_500yr.out') # Directory to save processed data
my_shelf = shelve.open(filename_out)
globals()['AMOC_max']=my_shelf['AMOC_max']
globals()['SMOC_min']=my_shelf['SMOC_min']
my_shelf.close()

AMOC_max_30S= AMOC_max[:,60] # AMOC at 30S # Max of streamfunction method
SMOC_min_30S= SMOC_min[:,60] # SMOC at 30S # Max of streamfunction method

np.nanmean(SMOC_min_30S) # SV # All years average

np.nanmean(SMOC_min_30S[R_ii_WS]) # SV # Convective years averge

np.nanmean(SMOC_min_30S[R_jj_WS]) # SV # Non-Convective years averge

np.nanmean(AMOC_max_30S) # SV # All years average

np.nanmean(AMOC_max_30S[R_ii_WS]) # SV # Convective years averge

np.nanmean(AMOC_max_30S[R_jj_WS]) # SV # Non-Convective years averge

# ACC flow (Sv) in the Drake passage
data_stream, transport_0_1000,transport_3000_4200,ii_max,jj_max, ii_min, jj_min  =calc_stream(dset_uo, year_start, year_end, mask, 1, [0,1000],[3000, 4200],0)
​
ACC_all = np.load('GFDL-ESM2G_ACC_500yr_data_stream_ACC.npy')
ACC_all_DRAKE_transport=np.nansum(ACC_all[:,25:35,295],axis=1)



